## This code identifies the top 10 Concepts pages and looks at what users do after they view the Concepts page.

In [ ]:
import os, pandas as pd

from weco_datascience.reporting import get_recent_data

In [ ]:
df = get_recent_data(config=os.environ, n=500000, index="metrics-conversion-prod")

In [ ]:
df=df.loc[(df["page.path"].str.contains("cachebust")==False)]

In [ ]:
print("Data has been taken from", df.tail(1)["@timestamp"])

In [ ]:
print("to", df.head(1)["@timestamp"]) 

### [Definitions of actions] (https://app.gitbook.com/o/-LumfFcEMKx4gYXKAZTQ/s/MM8zK9GZyFCpy8kvd5Km/analytics-and-reporting/kibana-definitions) are translated into code below

In [ ]:
df2 = df.fillna("")

concepts = df2.loc[(df["page.name"]=="concept") & (
df["properties.event"].isnull()) & (~
df["page.query.conceptId"].isnull()) & (
    df["page.path"].str.contains("#", "cachebust")==False)]
# note: when image modals trigger in open tabs, properties.event!="Open image modal"
concepts["action"]="concepts"
concepts_tabnav = df2.loc[(df["page.name"]=="concept") & (
df["properties.event"]=="Click tab nav") & (~
df["page.query.conceptId"].isnull()) & (
    df["page.path"].str.contains("#")==False)]
concepts_tabnav["action"]="concepts_tabnav"
concepts_imageview = df2.loc[(df["page.name"]=="concept") & (~
df["page.query.conceptId"].isnull()) & (
    df["properties.event"]=="Concepts open image modal")]
concepts_imageview["action"]="Concepts open image modal"
search = df.loc[(df["page.name"]=="works") & (df["page.query.query"].notnull())]
search["action"]="search"
works =  df.loc[(df["page.name"]=="work") & (df["page.query.workId"].notnull())]
works["action"]="works"
item_view = df.loc[(df["page.name"]=="item") & (
    df["properties.event"]!="download") & (df["source"]!="viewer/scroll")]
item_view["action"]="item_view"
image_view =  df.loc[(df["page.name"]=="image") & (df["type"]=="pageview")]
image_view["action"]="image_view"
image_modal = df.loc[(df["page.name"]=="images") & (
    df["properties.event"]=="Open image modal") & (df["page.query.query"].notnull())]
image_modal["action"]="image_modal"
exhibition = df.loc[(df["page.name"]=="exhibition")]
exhibition["action"]="exhibition"
story = df.loc[(df["page.name"]=="story")]
story["action"]="story"

In [ ]:
selected=[concepts, concepts_tabnav, concepts_imageview, search, works, item_view, image_view, image_modal, exhibition, story]
df2=pd.concat(selected)

In [ ]:
df3=df2[["page.path", "@timestamp", "anonymousId", "page.query.query", "page.query.workId", "page.name", "properties.event", "action", "page.query.conceptId"]]

In [ ]:
df3.to_csv("df3.csv")

### Finding top 10 Concepts pages

In [ ]:
pages=concepts[["page.query.conceptId","anonymousId"]]
pages["count"]=1
pages["sum"]= pages.groupby(pages["page.query.conceptId"])["count"].transform("sum")

In [ ]:
#drop duplicates and sort

top=pages.drop_duplicates("page.query.conceptId")
top2=top.sort_values("sum", ascending=False).head(10)
top2.to_csv("top2.csv")

### What did the users who viewed the top Concepts page do next?

In [ ]:
who=df3.loc[(df3["page.query.conceptId"]=="dp2rhtpm")]
who2=who["anonymousId"]
who2.drop_duplicates()
who3=who2.values.tolist()
concepts_behaviour=df3.loc[(df3["anonymousId"].isin(who3))]
concepts_behaviour.to_csv("concepts_behaviour.csv") 
len(who2)

In [ ]:
len(concepts_behaviour)

### sort and reindex by anonid and timestamp ascending

In [ ]:
cb2=concepts_behaviour.sort_values(["anonymousId","@timestamp"], ascending=[True,True])
cb3=cb2.reset_index()
cb3.to_csv("cb3.csv")

In [ ]:
data = []
anonid=[]

for i, row in cb3.iterrows():
    
    if (anonid==cb3.loc[i, "anonymousId"]): 
        data.append(row)
        
    if cb3.loc[i, "action"]=="concepts": 
        data.append(row)
        anonid=cb3.loc[i, "anonymousId"] 
        

results = pd.DataFrame(data)
results

In [ ]:
following=results.loc[(results["action"]!="concepts")]
following[["action", "anonymousId"]].groupby("action").count()
#len(following)

In [ ]:
dedupe=following["anonymousId"].drop_duplicates()
len(dedupe)

In [ ]:
results.to_csv("results.csv")